# 1.f

# 2. I'm not sure if the the errors are absolute or relative. If all of the scenarios are acceptable, I would choose scenario 1.
# This scenario does not have the best ratio between traning error and validation error, worse than scenario 2 only,
# but the complexity is less because the depth of the tree is only 2 (use Occam's razor principle)

# 3. AdaBoost reweighs the weights of observations to force the new learners focus on the observations misclassified by previous learners.
# Gradient Boosting uses the gradient of loss function to push the estimator to the parameter that achieve the minima of the cost function.

# 4. No. Depending on the adjustment of hyper-parameters, any model can overfit or underfit the dataset.

# 5.a Random forest does not have a learning rate parameter since the learners are dependent. AdaBoost did not have it in the orginal proposal but was implemented afterward.

In [1]:
## 6.a
import boto3
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'danhtran358-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining input and target variables
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

In [3]:
## function to get recall from random forest model
def get_scores_rf(X_train, X_test, Y_train, Y_test):
    ## Random forest model
    md_rf = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_rf = md_rf.predict_proba(X_test)[:,1]
    pred_rf = np.where(pred_rf < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_rf), accuracy_score(Y_test, pred_rf)

In [4]:
## function to get recall from AdaBoost model with Decision tree
def get_scores_ada(X_train, X_test, Y_train, Y_test):
    ## Adaboost model with Decision Tree leaners
    md_ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_ada = md_ada.predict_proba(X_test)[:,1]
    pred_ada = np.where(pred_ada < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_ada), accuracy_score(Y_test, pred_ada)

In [5]:
## function to get recall from AdaBoost model with SVC
def get_scores_grad(X_train, X_test, Y_train, Y_test):
    ## Adaboost model with Support Vector Machine leaners
    md_grad = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_grad = md_grad.predict_proba(X_test)[:,1]
    pred_grad = np.where(pred_grad < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_grad), accuracy_score(Y_test, pred_grad)

In [6]:
## 6.b
## list of scores
rf_recall_scores = list()
rf_accuracy_scores = list()
ada_recall_scores = list()
ada_accuracy_scores = list()
grad_recall_scores = list()
grad_accuracy_scores = list()


for i in range(100):
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ## get and store new scores to lists
    rf_recall, rf_accuracy = get_scores_rf(X_train, X_test, Y_train, Y_test)
    rf_recall_scores.append(rf_recall)
    rf_accuracy_scores.append(rf_accuracy)
    
    ada_recall, ada_accuracy = get_scores_ada(X_train, X_test, Y_train, Y_test)
    ada_recall_scores.append(ada_recall)
    ada_accuracy_scores.append(ada_accuracy)
    
    grad_recall, grad_accuracy = get_scores_grad(X_train, X_test, Y_train, Y_test)
    grad_recall_scores.append(grad_recall)
    grad_accuracy_scores.append(grad_accuracy)    
    
## print the average recall scores
print('Random Forest')
print('Average recall score:  ', np.mean(rf_recall_scores))
print('Average accuracy score:', np.mean(rf_accuracy_scores))
print('AdaBoost')
print('Average recall score:  ', np.mean(ada_recall_scores))
print('Average accuracy score:', np.mean(ada_accuracy_scores))
print('Gradient Boosting')
print('Average recall score:  ', np.mean(grad_recall_scores))
print('Average accuracy score:', np.mean(grad_accuracy_scores))

Random Forest
Average recall score:   0.8579464285714287
Average accuracy score: 0.4481830601092897
AdaBoost
Average recall score:   0.9914285714285715
Average accuracy score: 0.15849726775956283
Gradient Boosting
Average recall score:   0.8104464285714286
Average accuracy score: 0.5075683060109291


In [7]:
## I would use Gradient Boosting to predict.

In [11]:
## 6.c
gradient_recall_scores_train = list()
gradient_accuracy_scores_train = list()
gradient_recall_scores_test = list()
gradient_accuracy_scores_test = list()

for i in range(10):
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    md_gradient = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

    ## Predict and classfify train dataset
    pred_train_gradient = md_gradient.predict_proba(X_train)[:,1]
    pred_train_gradient = np.where(pred_train_gradient < 0.1, 0, 1)

    ## Predict and classfify test dataset
    pred_test_gradient = md_gradient.predict_proba(X_test)[:,1]
    pred_test_gradient = np.where(pred_test_gradient < 0.1, 0, 1)

    ## Calculate scores
    gradient_recall_scores_train.append(recall_score(Y_train, pred_train_gradient))
    gradient_accuracy_scores_train.append(accuracy_score(Y_train, pred_train_gradient))
    gradient_recall_scores_test.append(recall_score(Y_test, pred_test_gradient))
    gradient_accuracy_scores_test.append(accuracy_score(Y_test, pred_test_gradient))

print('Gradient Boosting')
print('Train dataset')
print('Average recall score:  ', np.mean(gradient_recall_scores_train))
print('Average accuracy score:', np.mean(gradient_accuracy_scores_train))
print('Test dataset')
print('Average recall score:  ', np.mean(gradient_recall_scores_test))
print('Average accuracy score:', np.mean(gradient_accuracy_scores_test))

Gradient Boosting
Train dataset
Average recall score:   0.8629213483146069
Average accuracy score: 0.5241108071135431
Test dataset
Average recall score:   0.8214285714285714
Average accuracy score: 0.5099726775956284


In [ ]:
## The recall and accuracy of train and test are similar, so generalization is good.
## The problem is train accuracy is low, the model seems underfit the dataset.
## I would increase the number of learners and adjust learning rate accordingly.